In [1]:
def obtener_tesis(driver):
    """Esta función recibe como argumento el driver, crea una lista vacia que será compuesta por
    todas las tesis de cada página"""
    
    #Creamos una lista vacía para que contenga todas las tesis 
    tesis_total = []
    # Obtenemos los impares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item odd"]'))
    # Obtenemos los pares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item even"]'))
    # Transformamos la lista de listas en una sola lista 
    tesis_total =[t for l in tesis_total for t in l]
    return tesis_total

In [2]:
# Creamos una lista vacía para almacenar el resultado de la siguiente función
info = []
def obtener_info(driver):
    """
    1. Esta función recibe como argumento el driver y ejecuta la función de obtener_tesis
    2. Hace un loop sobre las tesis obtenidas, extrayendo la fecha y el título
    3. Almacena la información en un diccionario 
    4. Adjunta el diccionario a la lista info creada al comienzo
    """
    
    tesis_totales = obtener_tesis(driver)
    print('iniciando scraping...')
    #info = []
    for tesis in tesis_totales:
        # Obtenemos fecha
        fecha = tesis.find_element_by_xpath('.//span[@class="date"]').text
        # Obtenemos título
        titulo = tesis.find_element_by_xpath('.//h4[@class="artifact-title"]').text
        info_dict={'Fecha': fecha, 'Título': titulo}
        info.append(info_dict)
    return info

In [3]:
def next_page(driver):
    """Esta función recibe como argumento el driver, encuentra el link para pasar la página y da click"""
    next_page = driver.find_element_by_xpath('//a[@class="next-page-link"]').click()

In [14]:
def main(driver, n):
    """
    Esta función recibe como argumento el driver y pasa por todas las páginas del repositorio,
    y hace un loop para obtener la información de todas las tesis
    """
    # Se divide el numero total de tesis sobre 20(numero de tesis x página) y se establece el rango del loop
    for x in range (1,n):
        # Ejecutamos la función para obtener la información de cada tesis
        obtener_info(driver)
        # Le damos click a la siguiente página  
        next_page(driver)   
    return info 

In [18]:
regex = r'(\d{1,4})'
def clean_date(df, regex):
    """
    Esta función recibe como input un dataframe y un patrón de expresión regular, para dejar
    en la fecha solo el año
    """

    df['Fecha'] = df['Fecha'].str.extract(regex).squeeze().str.zfill(4)
    return df

In [20]:
def clean_titulo():
    """Esta función recibe un Data Frame y a la columna con """
    # Definimos los signos de puntiación en una variable
    punt_espa = string.punctuation + "–¡¿”“•\r"
    
    # Definimos función para quitar los signos de puntiación 
    def sin_punt(x):
        x = ''.join([i for i in x if i not in (punt_espa)])
        return x
    
    # Aplicamos la función para quitar los signos
    df['Título cleaned'] = df['Título'].apply(sin_punt)
    
    # Definimos las stopwords
    stopwords_esp = stopwords.words('spanish')
    stopwords_esp = stopwords_esp + ['estudio', 'caso', 'análisis', 'bogotá', 'periodo', 'incidencia']
    
    # Ponemos las palabras todas en minúscula y las separamos
    df['Título cleaned'] = df['Título cleaned'].str.lower().str.split()
    
    # Eliminamos las stopwords
    df['Título cleaned'] = df['Título cleaned'].apply(lambda x: [item for item in x if item not in stopwords_esp])
    
    
    return df

In [8]:
def sort_df(df):
    """Esta función organiza de menor a mayor el index"""
    
    #Convierte el index de string a integrer
    df.index = df.index.astype(int)
    df = df.sort_index()
    return df

In [9]:
def find_ngrams(input_list, n):
    
    return list(zip(*[input_list[i:] for i in range(n)]))

In [17]:
def bigramas(df):
    """Esta función encuentra los bigramas por row dentro de una colomna seleccionada de un dataframe"""
    df['bigrams'] = df['Título cleaned'].map(lambda x: find_ngrams(x, 2))
    return df

In [11]:
# Bigram Frequency Counts
def freq_bigram(df):
    """
    Esta función recibe un dataframe y cuenta las veces que se repite cada bigrama en el texto
    """
    bigrama = df.tolist()
    bigrama = list(chain(*bigrama))
    bigrama = [(x.lower(), y.lower()) for x,y in bigrama]

    freq_bigrama = Counter(bigrams)
    return freq_bigrama